<a href="https://colab.research.google.com/github/rgabelarde/youtube-scrape/blob/main/Youtube_Scraper_(using_keyword_url).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YOUTUBE WEBSCRAPING**






---
 <br>

# **PRE-SCRAPING**

> Run these code before executing scraping below

<br>

##**Pre-requisite**##

>1. Excel file with a column of **either video urls (jump to 2)/keywords (jump to 3)**
2. Your excel file must be named "youtube_urls.xlsx"
3. Your excel file must be named "youtube_keywords.xlsx"

***Note: Please remember to run the code in order***

<br><br>


---




## **Installing and importing required packages for scraping**
>You MUST run this to successfully scrape data.

In [ ]:
# Installation of external packages selenium and youtube_transcript_api packages

!apt-get update # to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
!pip install selenium

!pip install youtube_transcript_api
!pip install youtube-search-python

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [61.8 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
# Import required packages
# Basic Python Packages
import asyncio
import re
import sys
import time
from urllib.request import urlopen
import os

# Google package to upload file
from google.colab import files

# External Packages
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver  # for webdriver
from selenium.common import exceptions
from selenium.webdriver.chrome.options import \
    Options  # for suppressing the browser
from youtube_transcript_api import YouTubeTranscriptApi
from youtubesearchpython import VideosSearch


<br>





---



<br>


## **Information Scraping**


>Initial scrapping method to scrap from keyword

In [ ]:
# Credits to @Jalexmercerind from GitHub for youtube-search-python api
# https://github.com/alexmercerind/youtube-search-python

def scrape_keyword_search(keyword, limit):
    # Scrape data
    videosSearch = VideosSearch(keyword, limit=limit)
    search_res = videosSearch.result()['result']

    # Convert dictionary results to dataframe
    res_df = pd.DataFrame(search_res)

    # Add a column that indicates keyword
    res_df['keyword'] = keyword

    # Delete some empty/unneeded columns
    # publishedTime replaced in other method for accuracy
    # viewCount, link replaced in another method for ease
    # description replaced in other method for completeness
    res_df.drop([
        'id', 'shelfTitle', 'accessibility', 'richThumbnail', 'publishedTime',
        'viewCount', 'descriptionSnippet', 'title'
    ],
      axis=1,
      inplace=True)
    
    return res_df

<br>

Scraping method helper to gather extra video information;
>- Used after scraping search results when scraping keyword data
- Used as method to scrape data from url


In [ ]:
# Credits to @faheel from GitHub for youtube-scraper-api
# https://github.com/faheel/youtube-scraper-api

def scrape_video_info(url):
    '''
    Scrapes data from the YouTube video's page whose ID is passed in the URL,
    and returns a JSON object as a response.
    '''

    # Set default values in case nothing is returned
    RESPONSE = {
        'id': "",
        'title': "",
        'url': url,
        'upload_date': "",
        'description': "",
        'genre': "",
        'user_id': "",
        # 'user_name': "",
        # 'is_user_verified': False,
        'views': 0,
        'likes': 0,
        'dislikes': 0,
        'is_paid': False,
        'is_unlisted': False,
        'is_family_friendly': True
    }
    

    # Helper functions
    def is_true(string):
        return string.lower() not in ['false', '0']

    def make_soup(url):
      '''
        Reads the contents at the given URL and returns a Python object based on
        the structure of the contents (HTML).
      '''
      html = urlopen(url).read()
      return BeautifulSoup(html, 'html.parser')

    soup = make_soup(url)

    # video_containers = soup.find_all('div', class_ = 'watch-main-col')
    soup_itemprop = soup.find(id='watch7-content')

    if len(soup_itemprop.text) > 1:
        video = RESPONSE

        # get data from tags having `itemprop` attribute
        for tag in soup_itemprop.find_all(itemprop=True, recursive=False):
            key = tag['itemprop']
            if key == 'videoId':
                # get video ID
                video['id'] = tag['content']
            elif key == 'name':
                # get video's name
                video['title'] = tag['content']
            elif key == 'datePublished':
                # get video's upload date
                video['upload_date'] = tag['content']
            elif key == 'genre':
                # get video's genre (category)
                video['genre'] = tag['content']
            elif key == 'paid':
                # is the video paid?
                video['is_paid'] = is_true(tag['content'])
            elif key == 'unlisted':
                # is the video unlisted?
                video['is_unlisted'] = is_true(tag['content'])
            elif key == 'isFamilyFriendly':
                # is the video family friendly?
                video['is_family_friendly'] = is_true(tag['content'])
            elif key == 'interactionCount':
                # get video's views
                video['views'] = int(tag['content'])
            elif key == 'channelId':
                # get uploader's channel ID
                video['user_id'] = tag['content']
            # elif key == 'author':
            #     # get uploader's channel name
            #     video['user_name'] = tag['content']
            elif key == 'description':
                video['description'] = tag['content']

        all_scripts = soup.find_all('script')
        for i in range(len(all_scripts)):
            try:
                if 'ytInitialData' in all_scripts[i].string:
                    match = re.findall(
                        "label(.*)",
                        re.findall("LIKE(.*?)like",
                                   all_scripts[i].string)[0])[0]
                    hasil = (''.join(match.split(',')).split("\"")[-1]).strip()
                    try:
                        video['likes'] = eval(hasil)
                    except:
                        video['likes'] = 0

                    match = re.findall(
                        "label(.*)",
                        re.findall("DISLIKE(.*?)dislike",
                                   all_scripts[i].string)[0])[0]
                    hasil = (''.join(match.split(',')).split("\"")[-1]).strip()
                    try:
                        video['dislikes'] = eval(hasil)
                    except:
                        video['dislikes'] = 0
                  
            except:
                pass
        return RESPONSE

    return ({'error': 'Video with the ID {} does not exist'.format(id)})




<br>





---



<br>


## **Comments Scraping**

In [ ]:
# Amended code from @dddat1017
# https://github.com/dddat1017/Scraping-Youtube-Comments

# Method to scrape youtube comments
# Output: dataframe
def scrape_comments(url):
    '''
    Extracts the comments from the Youtube video given by the URL.

    Args:
        url (str): The URL to the Youtube video

    Raises:
        selenium.common.exceptions.NoSuchElementException:
        When certain elements to look for cannot be found
    '''

    # Note: Download and replace argument with path to the driver executable.
    # Simply download the executable and move it into the webdrivers folder.
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=options)

    # Navigates to the URL, maximizes the current window, and
    # then suspends execution for (at least) 5 seconds (this
    # gives time for the page to load).
    driver.get(url)
    time.sleep(5)

    try:
        # Extract the elements storing the video title and
        # comment section.
        # title = driver.find_element_by_xpath('//*[@id='container']/h1/yt-formatted-string').text
        comment_section = driver.find_element_by_xpath('//*[@id="comments"]')
    except exceptions.NoSuchElementException:
        # Note: Youtube may have changed their HTML layouts for
        # videos, so raise an error for sanity sake in case the
        # elements provided cannot be found anymore.
        error = 'Error: Double check selector OR '
        error += 'element may not yet be on the screen at the time of the find operation'
        print(error)

    # Scroll into view the comment section, then allow some time
    # for everything to be loaded as necessary.
    driver.execute_script('arguments[0].scrollIntoView();', comment_section)
    time.sleep(7)

    # Scroll all the way down to the bottom in order to get all the
    # elements loaded (since Youtube dynamically loads them).
    last_height = driver.execute_script(
        'return document.documentElement.scrollHeight')

    while True:
        # Scroll down 'til 'next load'.
        driver.execute_script(
            'window.scrollTo(0, document.documentElement.scrollHeight);')

        # Wait to load everything thus far.
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script(
            'return document.documentElement.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height

    # One last scroll just in case.
    driver.execute_script(
        'window.scrollTo(0, document.documentElement.scrollHeight);')

    try:
        # Extract the elements storing the usernames and comments.
        username_elems = driver.find_elements_by_xpath(
            '//*[@id="author-text"]')
        comment_elems = driver.find_elements_by_xpath(
            '//*[@id="content-text"]')
    except exceptions.NoSuchElementException:
        error = 'Error: Double check selector OR '
        error += 'element may not yet be on the screen at the time of the find operation'
        print(error)

    comments_df = pd.DataFrame({
        'commenter_username': [username.text for username in username_elems],
        'comment_text': [comment.text for comment in comment_elems],
    })

    driver.close()

    return comments_df


<br>





---



<br>


## **Transcript Scraping**


In [ ]:
def scrape_transcript(url):
    video_id = url.split('watch?v=')[1]
    try:
        transcripts = YouTubeTranscriptApi.get_transcript(video_id)

        transcript_df = pd.DataFrame({
            'transcribed_text':
            [transcripts['text'] for transcripts in transcripts],
            'start_time': [transcripts['start'] for transcripts in transcripts],
            'duration': [transcripts['duration'] for transcripts in transcripts],
        })
        
        return transcript_df
    except:
      return "No transcriptions for this video could be found"


<br>





---



<br>


## **Helper for Comments and Transcriptions Scraping**

>- Helps create an outputs folder
-  Calls both scraping functions
- Saves output as excel file with different sheets for comments and transcript

In [ ]:
# Create folder to store all comments and transcript files

output_path = 'comments and transcript outputs'

if not os.path.exists(output_path):
  os.makedirs(output_path)

In [ ]:
def scrape_comments_transcript(url, title):
  comments_df = scrape_comments(url)
  transcript_df = scrape_transcript(url)
  
  writer = pd.ExcelWriter(os.path.join(output_path, 
                          title + ".xlsx"), engine='openpyxl')
  
  if isinstance(comments_df, pd.DataFrame): 
    comments_df.to_excel(writer, sheet_name='comments', index=False)
  if isinstance(transcript_df, pd.DataFrame):
    transcript_df.to_excel(writer, sheet_name='transcript', index=False)

  writer.save()


<br>





---



<br>


## **Final Cleaning process**
>Used after keyword scrapping process

In [ ]:
def clean_keywords_data(df):
    # NTS: can use json_normalize because the dictionaries are flat
    # then drop initial clustered (dictionary-ed) column
    # Expand 'more_info' column that is currently a dictionary of data into columns
    df = df.join(pd.json_normalize(df.more_info)).drop('more_info', axis=1)
      

    # Expand 'video_creator' column that is currently a dictionary of data into columns
    df = df.join(pd.json_normalize(df.channel)['name']).drop(
        'channel', axis=1)
    
    # Rename columns accordingly for literacy
    df.rename(
        columns={
            'thumbnails': 'thumbnail_urls',
            'name': 'user_name'
        },
        inplace=True
    )

    # Delete empty/unneeded columns
    # link replaced in another method for ease
    df.drop(['link'], axis=1, inplace=True)

    df['thumbnail_urls'] = df['thumbnail_urls'].apply(
        lambda lst: ', \n'.join([item['url'] for item in lst]))

    # Rearrange columns for readability
    df = df[[
        'type', 'id', 'title', 'url', 'thumbnail_urls','keyword', 'description',
        'duration', 'upload_date', 'genre', 'user_id', 'user_name',
        'views', 'likes', 'dislikes', 'is_paid', 'is_unlisted', 'is_family_friendly'
    ]]

    return df


 <br> <br>


---
 <br>

# **KEYWORDS SCRAPING**
>Run this only if you are doing keyword scraping!

Note: Remember that your file name has to be named '**youtube_keywords.xlsx**'

### Video data scraping

In [ ]:
# Upload excel file here
uploaded = files.upload()

Saving youtube_keywords.xlsx to youtube_keywords.xlsx


In [ ]:
# Read in your file to a dataframe
keywords_df = pd.read_excel(uploaded.get('youtube_keywords.xlsx'))

# Enter the number of results you wish to fetch:
search_limit = int(input("How much data would you like to fetch per keyword? \n Please only enter an integer.\n"))

How much data would you like to fetch per keyword? 
 Please only enter an integer.
4


In [ ]:
# Convert keywords into a list
keywords = list(keywords_df['keyword'])

# Map each keyword to function that will scrape it's search results
df_lst = [scrape_keyword_search(keyword, search_limit) for keyword in keywords]

# Join scraped data from each keyword scraped
keywords_res_df = pd.concat(df_lst, ignore_index=True)

# Scrape extra information about each video
keywords_res_df['more_info'] = keywords_res_df['link'].apply(scrape_video_info)

# Clean all the data we scraped
keywords_res_df = clean_keywords_data(keywords_res_df)

In [ ]:
# Preview data to check
keywords_res_df.head()

,type,id,title,url,thumbnail_urls,keyword,description,duration,upload_date,genre,user_id,user_name,views,likes,dislikes,is_paid,is_unlisted,is_family_friendly
0,video,1HygThMLzGs,🤣Funny Dog Videos 2020🤣 🐶 It's time to LAUGH w...,https://www.youtube.com/watch?v=1HygThMLzGs,https://i.ytimg.com/vi/1HygThMLzGs/hq720.jpg?s...,dog,🤣Funny Dog Videos 2020🤣 🐶 It's time to LAUGH w...,9:56,2020-05-05,Pets & Animals,UCIqWFhsJm2VU1ZuGrZnx3Pw,MAI PM,28928713,349782,15106,False,False,True
1,video,ewJyUcL91oo,"Wasabi, the Pekingese, crowned Best in Show at...",https://www.youtube.com/watch?v=ewJyUcL91oo,https://i.ytimg.com/vi/ewJyUcL91oo/hq720.jpg?s...,dog,A champion was crowned at the 145th Westminste...,8:32,2021-06-13,Sports,UCwNqHDsnBCKT-olwJwIFyfg,FOX Sports,9977,132,9,False,False,True
2,video,gpyB54lSpYg,Dog Reviews Food With Baby Puppy | Tucker Tast...,https://www.youtube.com/watch?v=gpyB54lSpYg,https://i.ytimg.com/vi/gpyB54lSpYg/hq720.jpg?s...,dog,Dog Reviews Food With Baby Puppy | Tucker Tast...,3:11,2020-12-13,Pets & Animals,UCNSzfesc7IgWZwg4n6uXr1A,Tucker Budzyn,25020293,468935,16841,False,False,True
3,video,spjSnOuOxUw,TV for Dogs! Entertainment for Dogs and Puppie...,https://www.youtube.com/watch?v=spjSnOuOxUw,https://i.ytimg.com/vi/spjSnOuOxUw/hqdefault_l...,dog,TV for Dogs! Entertainment for Dogs and Puppie...,None,2021-03-22,Pets & Animals,UCVqytdh_eTzWlh1K8Yst_rg,Calm Your Dog - Relaxing Music and TV for Dogs,307027,935,63,False,False,True
4,video,eX2qFMC8cFo,Funniest Cats 😹 - Don't try to hold back Laugh...,https://www.youtube.com/watch?v=eX2qFMC8cFo,https://i.ytimg.com/vi/eX2qFMC8cFo/hq720.jpg?s...,cat,Funniest Cats 😹 - Don't try to hold back Laugh...,9:10,2020-10-29,Pets & Animals,UCYPrd7A27nLhQONcCIfFTaA,Funny Cats Life,47204786,761635,40301,False,False,True


In [ ]:
# export data to excel sheet
keywords_res_df.to_excel(r'youtube_keywords_output.xlsx', index=False)

### Video comments + transcript scraping

In [ ]:
keywords_res_df.apply(lambda x: scrape_comments_transcript(x['url'], x['title']), axis=1)

KeyboardInterrupt: ignored

 <br>

**end of keyword scraping**





---
 <br>

# **URLS SCRAPING**
>Run this only if you are doing url scraping!

Note: Remember that your file name has to be named '**youtube_urls.xlsx**'

### Video data scraping

In [ ]:
# Upload excel file here
uploaded = files.upload()

Saving youtube_urls.xlsx to youtube_urls.xlsx


In [ ]:
# Read in your file to a dataframe
urls_df = pd.read_excel(uploaded.get('youtube_urls.xlsx'))

In [ ]:
# Convert keywords into a list
urls = list(urls_df['URL'])

# Map each url to function that will scrape its video's data
urls_res_df = pd.DataFrame(map(scrape_video_info, urls))

In [ ]:
# Preview data to check
urls_res_df.head()

,id,title,url,upload_date,description,genre,user_id,views,likes,dislikes,is_paid,is_unlisted,is_family_friendly
0,Ab2hF-EN1lw,Jake Scott - Anymore (Audio),https://www.youtube.com/watch?v=Ab2hF-EN1lw,2021-06-10,"Stream ""Anymore"" here: http://smarturl.it/jake...",Music,UCDvr1r2d2Ma4GrP8rKP8HjA,25702,2487,10,False,False,True
1,Kz2MLCCdxsM,Jake Scott - Off (Lyric Video),https://www.youtube.com/watch?v=Kz2MLCCdxsM,2021-04-30,"Stream ""Off"" here: http://smarturl.it/Offjakes...",Music,UCDvr1r2d2Ma4GrP8rKP8HjA,162597,8464,34,False,False,True
2,zz8lKer83Ac,"Jake Scott & Josie Dunne - ""CWJBHN"" (Lyric Video)",https://www.youtube.com/watch?v=zz8lKer83Ac,2020-07-16,"Jake Scott music, merch & more: https://solo.t...",Music,UCDvr1r2d2Ma4GrP8rKP8HjA,2379153,94933,435,False,False,True
3,HaBOp-9pDDQ,Jake Scott - Meet My Shadow (Audio Video),https://www.youtube.com/watch?v=HaBOp-9pDDQ,2021-05-20,"Stream ""Meet My Shadow"" here: http://smarturl....",Music,UCDvr1r2d2Ma4GrP8rKP8HjA,45579,2762,6,False,False,True


In [ ]:
# export data to excel sheet
urls_res_df.to_excel(r'youtube_urls_output.xlsx', index=False)

### Video comments + transcript scraping

In [ ]:
urls_res_df.apply(lambda x: scrape_comments_transcript(x['url'], x['title']), axis=1)

0    None
1    None
2    None
3    None
dtype: object

 <br>

**end of url scraping**





---

<br><br>